In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [18]:
examples = [
    {
        "question": "당신은 프랑스에 대해서 무엇을 알고있나요?",
        "answer": """
        내가 알고있는것:
        수도: 파리
        언어: 불어
        음식: 와인과 치즈
        통화: 유로
        """,
    },
    {
        "question": "당신은 이탈리아에 대해서 무엇을 알고있나요?",
        "answer": """
        내가 알고있는 것:
        수도: 로마
        언어: 이탈리아어
        음식: 피자와 파스타
        통화: 유로
        """,
    },
    {
        "question": "당신은 그리드에 대해서 무엇을 알고있나요?",
        "answer": """
        내가 알고있는 것은:
        수도: 아테네
        언어: 그리스어
        음식: 수플라키와 페타치즈
        통화: 유로
        """,
    },
]

examples_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(examples_template)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: 당신이 {country}에 대해서 무엇을 알고있나요?",
    input_variables=["country"],
)

chain = prompt | chat
chain.invoke({"country": "터키"})

AI: 
    내가 알고있는 것:
    수도: 안카라
    언어: 터키어
    음식: 케밥과 바클라바
    통화: 터키 리라

AIMessageChunk(content='AI: \n    내가 알고있는 것:\n    수도: 안카라\n    언어: 터키어\n    음식: 케밥과 바클라바\n    통화: 터키 리라')